In [30]:
# importing required libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_log_error as msle

In [31]:
final_data = pd.read_csv('Downloads/B1G6ipV5TZsnUOqlBY1Q_4 FeatureEngineeringDatasetandnotebooks-191226-111656/4 FeatureEngineeringDatasetandnotebooks-191226-111656/final_data.csv')

In [32]:
final_data.head()

,WEEK_END_DATE,STORE_NUM,UPC,BASE_PRICE,FEATURE,DISPLAY,UNITS,MANUFACTURER_1,MANUFACTURER_2,MANUFACTURER_3,...,week,quarter,U_MANUFACTURER,U_CATEGORY,U_SUB_CATEGORY,UNITS_BEFORE_52WEEK,price_difference,2_MONTH_BEFORE,1_WEEK_BEFORE,AVERAGE_UNITS_IN_2_MONTH
0,2009-01-14,367,1111009477,1.57,0,0,13,1,0,0,...,3,1,4,3,5,0.0,0.0,2008-11-12,2009-01-07,0.0
1,2009-01-14,367,1111009497,1.39,0,0,20,1,0,0,...,3,1,4,3,5,0.0,0.0,2008-11-12,2009-01-07,0.0
2,2009-01-14,367,1111085319,1.88,0,0,14,1,0,0,...,3,1,4,3,5,0.0,0.0,2008-11-12,2009-01-07,0.0
3,2009-01-14,367,1111085345,1.88,0,0,29,1,0,0,...,3,1,4,3,5,0.0,0.0,2008-11-12,2009-01-07,0.0
4,2009-01-14,367,1111085350,1.98,0,0,35,1,0,0,...,3,1,4,3,5,0.0,0.0,2008-11-12,2009-01-07,0.0


In [33]:
final_data.columns

Index(['WEEK_END_DATE', 'STORE_NUM', 'UPC', 'BASE_PRICE', 'FEATURE', 'DISPLAY',
       'UNITS', 'MANUFACTURER_1', 'MANUFACTURER_2', 'MANUFACTURER_3',
       'MANUFACTURER_4', 'MANUFACTURER_5', 'MANUFACTURER_6', 'MANUFACTURER_7',
       'MANUFACTURER_8', 'MANUFACTURER_9', 'CATEGORY_1', 'CATEGORY_2',
       'CATEGORY_3', 'CATEGORY_4', 'SUB_CATEGORY_1', 'SUB_CATEGORY_2',
       'SUB_CATEGORY_3', 'SUB_CATEGORY_4', 'SUB_CATEGORY_5', 'SUB_CATEGORY_6',
       'SUB_CATEGORY_7', 'PRODUCT_SIZE', 'STORE_ID',
       'ADDRESS_STATE_PROV_CODE_1', 'ADDRESS_STATE_PROV_CODE_2',
       'ADDRESS_STATE_PROV_CODE_3', 'ADDRESS_STATE_PROV_CODE_4', 'MSA_CODE_1',
       'MSA_CODE_2', 'MSA_CODE_3', 'MSA_CODE_4', 'MSA_CODE_5', 'MSA_CODE_6',
       'MSA_CODE_7', 'MSA_CODE_8', 'MSA_CODE_9', 'SEG_VALUE_NAME',
       'SALES_AREA_SIZE_NUM', 'AVG_WEEKLY_BASKETS', 'year', 'month', 'day',
       'day_of_year', 'week', 'quarter', 'U_MANUFACTURER', 'U_CATEGORY',
       'U_SUB_CATEGORY', 'UNITS_BEFORE_52WEEK', 'price_diffe

In [34]:
final_data['WEEK_END_DATE'] = pd.to_datetime(final_data['WEEK_END_DATE'])

In [35]:
final_data['day_of_week'] = final_data['WEEK_END_DATE'].dt.dayofweek  # 0 = Monday, 6 = Sunday
final_data['week_of_year'] = final_data['WEEK_END_DATE'].dt.isocalendar().week
final_data['month'] = final_data['WEEK_END_DATE'].dt.month
final_data['quarter'] = final_data['WEEK_END_DATE'].dt.quarter
final_data['is_weekend'] = final_data['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
final_data['day_of_month'] = final_data['WEEK_END_DATE'].dt.day
final_data['year'] = final_data['WEEK_END_DATE'].dt.year
final_data['season'] = final_data['month'].apply(lambda x: (x%12 + 3)//3)  # 1: Winter, 2: Spring, 3: Summer, 4: Fall
final_data['is_holiday'] = final_data['WEEK_END_DATE'].apply(lambda x: 1 if x in ['2023-12-25', '2024-01-01'] else 0)  # Example holidays
final_data['days_since_start'] = (final_data['WEEK_END_DATE'] - final_data['WEEK_END_DATE'].min()).dt.days

In [36]:
final_data['store_product_avg_sales'] = final_data.groupby(['STORE_ID', 'UPC'])['BASE_PRICE'].transform('mean')
final_data['store_total_sales'] = final_data.groupby('STORE_ID')['BASE_PRICE'].transform('sum')
final_data['store_product_sales_ratio'] = final_data['BASE_PRICE'] / final_data['price_difference']
final_data['store_avg_revenue'] = final_data.groupby('STORE_ID')['season'].transform('mean')
final_data['store_sales_volatility'] = final_data.groupby('STORE_ID')['BASE_PRICE'].transform('std')
final_data['store_customer_count'] = final_data.groupby('STORE_ID')['CATEGORY_1'].transform('sum')  # Assuming customer_count exists
final_data['store_avg_discount'] = final_data.groupby('STORE_ID')['SUB_CATEGORY_5'].transform('mean')
final_data['store_product_diversity'] = final_data.groupby('STORE_ID')['BASE_PRICE'].nunique()

In [37]:
final_data['product_avg_sales'] = final_data.groupby('FEATURE')['BASE_PRICE'].transform('mean')
final_data['product_sales_volatility'] = final_data.groupby('FEATURE')['BASE_PRICE'].transform('std')
final_data['product_lifetime_avg_sales'] = final_data.groupby('FEATURE')['BASE_PRICE'].expanding().mean().reset_index(level=0, drop=True)
final_data['product_discount'] = 1 - (final_data['BASE_PRICE'] / final_data['price_difference'])  # Assuming price columns exist
final_data['product_seasonality'] = final_data.groupby(['FEATURE', 'season'])['BASE_PRICE'].transform('mean')
final_data['product_store_diversity'] = final_data.groupby('FEATURE')['STORE_ID'].nunique()
final_data['product_price_fluctuation'] = final_data.groupby('FEATURE')['price_difference'].transform('std')

In [38]:
final_data = final_data.drop(columns=['WEEK_END_DATE','2_MONTH_BEFORE','1_WEEK_BEFORE'])

In [39]:
from sklearn.model_selection import train_test_split


In [40]:
target = 'BASE_PRICE'
features = [col for col in final_data.columns if col != target]
X_train, X_valid, y_train, y_valid = train_test_split(final_data[features], final_data[target], test_size=0.2, random_state=42)

In [41]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

ValueError: Input X contains infinity or a value too large for dtype('float32').

In [42]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)


ValueError: Input X contains infinity or a value too large for dtype('float64').

In [43]:
print(np.isinf(X_train).sum().sum(), "infinite values in X_train")
print(np.isnan(X_train).sum().sum(), "NaN values in X_train")
print(np.isinf(X_valid).sum().sum(), "infinite values in X_valid")
print(np.isnan(X_valid).sum().sum(), "NaN values in X_valid")


238542 infinite values in X_train
371559 NaN values in X_train
59602 infinite values in X_valid
92895 NaN values in X_valid


In [44]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)  # Convert inf to NaN
X_valid.replace([np.inf, -np.inf], np.nan, inplace=True)


In [45]:
X_train.fillna(X_train.mean(), inplace=True)
X_valid.fillna(X_valid.mean(), inplace=True)


In [46]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)


In [47]:
print(np.isinf(X_train).sum().sum(), "infinite values in X_train after cleaning")
print(np.isnan(X_train).sum().sum(), "NaN values in X_train after cleaning")


0 infinite values in X_train after cleaning
0 NaN values in X_train after cleaning


In [48]:
rf_model.fit(X_train, y_train)


RandomForestRegressor(n_jobs=-1, random_state=42)

In [50]:
from sklearn.metrics import mean_absolute_error


In [51]:
y_pred = rf_model.predict(X_valid)
mae = mean_absolute_error(y_valid, y_pred)
print(f'Mean Absolute Error after Feature Engineering: {mae:.2f}')

Mean Absolute Error after Feature Engineering: 0.02
